# Step 10 - The Bull Strategy

Vikram is also similar to Leah but purchases Bitcoin frequently because he believes the price will go up in the future!

## Vikram's Transaction History
- Vikram also purchases 50 units of both ETH and BTC just like Leah on Jan 1st 2017
- He continues to purchase more throughout the entire 4 year period
- He does not sell any of his crypto - he's in it for the long run

## Vikram's Data
Because this is also a simplified version of our dataset - we will work on another cte table called `vikram_bull_strategy` with our data for these questions.

In [1]:
# Importing packages:
import pandas as pd
import os
import sqlalchemy

# Connecting to MySQL database:
host = os.environ.get('mysql_host')
user = os.environ.get('mysql_user')
password = os.environ.get('mysql_password')
engine = sqlalchemy.create_engine(f'mysql+pymysql://{user}:{password}@{host}/trading')

In [5]:
pd.read_sql(
    """
    SELECT *
    FROM transactions
    WHERE member_id='6512bd' 
      AND txn_type='BUY'
    LIMIT 10;
    """, 
    engine)

,txn_id,member_id,ticker,txn_date,txn_time,txn_type,quantity,percentage_fee
0,11,6512bd,BTC,2017-01-01,2017-01-01 00:00:00,BUY,50.00000,0.30
1,25,6512bd,ETH,2017-01-01,2017-01-01 00:00:00,BUY,50.00000,0.30
2,30,6512bd,ETH,2017-01-01,2017-01-01 06:22:20,BUY,8.84299,0.30
3,31,6512bd,BTC,2017-01-01,2017-01-01 06:40:48,BUY,2.27106,0.21
4,35,6512bd,BTC,2017-01-01,2017-01-01 11:00:14,BUY,6.73842,0.30
5,36,6512bd,BTC,2017-01-01,2017-01-01 12:03:33,BUY,9.37876,0.30
6,55,6512bd,BTC,2017-01-02,2017-01-02 11:12:42,BUY,5.54384,0.30
7,63,6512bd,ETH,2017-01-02,2017-01-02 20:48:13,BUY,5.04373,0.07
8,65,6512bd,BTC,2017-01-02,2017-01-02 21:00:49,BUY,3.01276,0.30
9,99,6512bd,ETH,2017-01-04,2017-01-04 22:04:12,BUY,1.83100,0.30


## Required Metrics
To assess Vikram's performance we also need to regularly match the prices for his trades throughout the 4 years and not just at the start of the entire dataset, like in the case of Leah's HODL strategy.

We will need to calculate the following metrics:

- Total investment amount in dollars for all of his purchases
- The dollar amount of fees paid
- The dollar cost average per unit of BTC and ETH purchased by Vikram
- The final investment value of his portfolio on August 29th 2021
- Profitability can be measured by final portfolio value divided by the investment amount
- Profitability split by BTC and ETH

## Solutions

**Question 1 & 2**

Calculate the total investment amount in dollars for all of Vikram's purchases and his dollar amount of fees paid

In [9]:
pd.read_sql_query(
    """
    SELECT *
    FROM prices
    LIMIT 5;
    """,
    engine
)

,ticker,market_date,price,open,high,low,volume,change
0,ETH,2021-08-29,3177.84,3243.96,3282.21,3162.79,582.04K,-2.04%
1,ETH,2021-08-28,3243.90,3273.78,3284.58,3212.24,466.21K,-0.91%
2,ETH,2021-08-27,3273.58,3093.78,3279.93,3063.37,839.54K,5.82%
3,ETH,2021-08-26,3093.54,3228.03,3249.62,3057.48,118.44K,-4.17%
4,ETH,2021-08-25,3228.15,3172.12,3247.43,3080.70,923.13K,1.73%


In [13]:
pd.read_sql(
    """
    WITH vikram_bull_strategy AS (
        SELECT *
        FROM transactions
        WHERE member_id='6512bd' 
          AND txn_type='BUY'
    )

    SELECT 
      SUM(v.quantity * p.price) AS total_investments,
      SUM(v.percentage_fee * v.quantity * p.price / 100) AS fees_paid
    FROM vikram_bull_strategy v
    INNER JOIN prices p
      ON v.txn_date=p.market_date
      AND v.ticker=p.ticker
    """, 
    engine)

,total_investments,fees_paid
0,5.073045e+07,128821.145294


**Question 3**
What is the average cost per unit of BTC and ETH purchased by Vikram

In [18]:
pd.read_sql(
    """
    WITH vikram_bull_strategy AS (
        SELECT *
        FROM transactions
        WHERE member_id='6512bd' 
          AND txn_type='BUY'
    )

    SELECT 
      v.ticker,
      SUM(v.quantity * p.price) / SUM(v.quantity) AS average_price
    FROM vikram_bull_strategy v
    INNER JOIN prices p
      ON v.txn_date=p.market_date
      AND v.ticker=p.ticker
    GROUP BY v.ticker
    ORDER BY average_price DESC;
    """, 
    engine)

,ticker,average_price
0,BTC,12190.138446
1,ETH,538.402091


**Question 4**

Calculate profitability by using final portfolio value divided by the investment amount

In [23]:
pd.read_sql(
    """
    WITH vikram_bull_strategy AS (
        SELECT *
        FROM transactions
        WHERE member_id='6512bd' 
          AND txn_type='BUY'
    ),

    cte_portfolio_values AS (
        SELECT
          SUM(t.quantity * p.price) AS initial_investment,
          SUM(t.quantity * f.price) AS final_value
        FROM vikram_bull_strategy AS t
        INNER JOIN prices AS p
          ON t.ticker = p.ticker
          AND t.txn_date = p.market_date
        INNER JOIN prices AS f
          ON t.ticker = f.ticker
        WHERE f.market_date = '2021-08-29'
    )

    SELECT
      final_value / initial_investment AS profitability
    FROM cte_portfolio_values;
    """, 
    engine)

,profitability
0,4.019205


**Question 5**

Calculate Vikram's profitability split by BTC and ETH

In [27]:
pd.read_sql(
    """
    WITH vikram_bull_strategy AS (
        SELECT *
        FROM transactions
        WHERE member_id='6512bd' 
          AND txn_type='BUY'
    ),

    cte_portfolio_values AS (
        SELECT
          t.ticker,
          SUM(t.quantity * p.price) AS initial_investment,
          SUM(t.quantity * f.price) AS final_value
        FROM vikram_bull_strategy AS t
        INNER JOIN prices AS p
          ON t.ticker = p.ticker
          AND t.txn_date = p.market_date
        INNER JOIN prices AS f
          ON t.ticker = f.ticker
        WHERE f.market_date = '2021-08-29'
        GROUP BY t.ticker
    )

    SELECT
      ticker,
      final_value / initial_investment AS profitability
    FROM cte_portfolio_values;
    """, 
    engine)

,ticker,profitability
0,ETH,5.902355
1,BTC,3.958528


# References

- [Data With Danny Course - Step 10](https://github.com/DataWithDanny/sql-masterclass/blob/main/course-content/step10.md)